### Long-term fitness trajectories from the LTEE

Data available in the Supplemental Material of Good et al. Nature 2017. 
Download possible from Ben Good's github repository [here](https://github.com/benjaminhgood/LTEE-metagenomic/blob/master/additional_data/Concatenated.LTEE.data.all.csv)

We follow the procedures from Wiser et al. 2013 [here](https://doi.org/10.1126/science.1243357). From the Supplemental Material, we are given the following information. 


- Summarizing statistical procedures to fit the two models
- Models were fit to fitness trajectories using the ‘nls’ package in r. 
- Model fits were compared using the BIC information criterion scores. These were then converted into an odds ratio. 
    - Table S1 shows the BIC scores and odds ratios for fits to subsets of the data: a) all 12 populations and all time points, b) excluding 3 populations with incomplete trajectories and c) excluding 6 populations that evolved hypermutability
    - Table S2 summarizes BIC scores for fits to individual populations. This also indicates if the population was truncated or a hypermutator 
    - Table S4 lists the estimated parameters for the power law fit

On the bigger picture, there is also the talk from 2013 by Wiser on [Youtube](https://www.youtube.com/watch?v=CmyBn5Cezy4) with 127 views as of September 2022. 

In [ ]:
### load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from scipy.optimize import curve_fit

In [ ]:
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/LTEE_fit/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:
df = pd.read_csv('./data/Concatenated.LTEE.data.all.csv')

In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())

### explanation of column headers and labels

    607 is the wild-type strain 'REL607'
    D.0 is the dilution factor applied to count colonies in initial inoculum. 
    D.1 is the dilution factor applied to count colonies in the saturated population. We expect D.1 = 100*D.0. 
    

In [ ]:
df.head(2)

In [ ]:

### split into two subsets, according to the Ara-marker of evolving population
is_Ara_positive = np.array(['+' in v for v in df['Population'].values])

df_pos = df[is_Ara_positive]
df_neg = df[~is_Ara_positive] ### only use Ara negative lineages

In [ ]:
## treat Ara-posative population
assert all(df_pos['Red.Pop'] == '606') # wild-type is always the red population

### re-construct population sizes
df_pos['Nwt.0'] = df_pos['Red.0']*df_pos['D.0']
df_pos['Nmut.0'] = df_pos['White.0']*df_pos['D.0']
df_pos['Nwt.1'] = df_pos['Red.1']*df_pos['D.1']
df_pos['Nmut.1'] = df_pos['White.1']*df_pos['D.1']



In [ ]:
## treat Ara-negative population
assert all(df_neg['White.Pop'] == '607') # wild-type is always the white population

### re-construct population sizes
df_neg['Nwt.0'] = df_neg['White.0']*df_neg['D.0']
df_neg['Nmut.0'] = df_neg['Red.0']*df_neg['D.0']
df_neg['Nwt.1'] = df_neg['White.1']*df_neg['D.1']
df_neg['Nmut.1'] = df_neg['Red.1']*df_neg['D.1']


In [ ]:
### join

df = df_pos.append(df_neg)

## reconstruct frequencies
df['xmut.0'] = df['Nmut.0']/(df['Nwt.0'] + df['Nmut.0'])
df['xmut.1'] = df['Nmut.1']/(df['Nwt.1']  + df['Nmut.1'])

In [ ]:
### reconstruct fitness statistics
df['s'] = np.log(df['Nmut.1']/df['Nwt.1']) - np.log(df['Nmut.0']/df['Nwt.0'])
df['W'] = np.divide( np.log(df['Nmut.1']/df['Nmut.0']),\
                            np.log(df['Nwt.1']/df['Nwt.0']))
df['delta_log'] = np.log(df['xmut.1']) - np.log(df['xmut.0'])

In [ ]:
### check that  my number is consistent with existing value for 'Fitness' in the dataset
np.allclose(df['W'], df['Fitness'],equal_nan=True)

In [ ]:
## manual check
df['gap'] = df['W'] - df['Fitness']
print(df['gap'].max())


In [ ]:
color_s = 'tab:grey'
color_W = 'firebrick'
color_deltalog = 'navy'


In [ ]:
color_hyper = 'tab:red'
def hyperbolic(t, a, b):
    ## compare first Equation in paper
    return 1 + np.divide(a*t,t+b)

color_power = 'tab:blue'
def powerlaw(t, a, b):
    ## compare second Equation in paper
    return np.power(b*t + 1,a)

In [ ]:
## we drop some superfluous columns
columns_auxiliary = [ 'Red.0', 'White.0', 'Red.1', 'White.1', 'D.0', 'D.1', 'gap', 'White.Pop', 'Red.Pop', 'Fitness', 'Nwt.0', 'Nmut.0', 'Nwt.1', 'Nmut.1', 'Nwt.0']

df = df.drop(columns_auxiliary, axis = 1)

In [ ]:
## shift data points for alternative statistics, which are based at zero
df['s+1'] = df['s'] +1
df['delta_log+1'] = df['delta_log'] +1

### reproduce fits to individual subpopulations

In [ ]:
list_populations = list(set(df['Population'].values))

In [ ]:
list_populations.sort()

In [ ]:
### set up Dataframe to store results
df_results = pd.DataFrame()
for pop in list_populations:
    df_tmp= pd.DataFrame(data = {'pop':3*[pop], 'target':['W', 's+1', 'delta_log+1']})
    df_results = df_results.append(df_tmp)
    
df_results = df_results.set_index(['pop','target'])


In [ ]:
(1,2) + tuple([3])

In [ ]:
%%time 

for target in ['W', 's+1', 'delta_log+1']: 
    fig, axes = plt.subplots(4,3, figsize = (4*FIGHEIGHT_TRIPLET,3*FIGWIDTH_TRIPLET) , sharex=True, sharey=True)
                         
    axes = axes.flatten()
    
    for pop,ax in zip(list_populations,axes):

        df_subset = df[df['Population'] == pop]
        ### remove na values
        df_subset= df_subset[~df_subset[target].isna()]
        df_results.at[(pop,target), 'n_datapoints'] = df_subset.shape[0] # store nubmer of datapoints
        df_results.at[(pop, target), 'Generation_max'] = df_subset['Generation'].max() # store final timepoint

        t = df_subset['Generation']
        y = df_subset[target]
        ## plot raw data
        sns.scatterplot(x=t,y=y, ax=ax, color = 'grey')
        sns.lineplot(x=t,y=y, ax =ax, color = 'grey')
        
        ## fit hyperbolic model
        popt_hyperbolic,_ = curve_fit(f=hyperbolic, xdata=t, ydata=y)
        ## store
        df_results.at[(pop,target), 'hyper_a'] = popt_hyperbolic[0]
        df_results.at[(pop,target), 'hyper_b'] = popt_hyperbolic[1]
        ## compute trajectory
        y_hat = hyperbolic(t, *popt_hyperbolic)
        ## store sum of residuals squared
        rss = np.power(y_hat - y,2).sum()
        rsquared = 1 - (rss/np.power(y - y.mean(),2).sum())
        df_results.at[(pop, target), 'hyper_rss']  = rss
        df_results.at[(pop, target), 'hyper_rsquared']  = rsquared
        ## plot trajectory
        param_label = 'hyperbolic: a=%.2f, b=%.0f' % tuple(popt_hyperbolic)
        sns.lineplot(x=t, y =y_hat, color = color_hyper,ax=ax,
                     label = param_label + rf", $R^2={rsquared:.2f}$")

        
        # fit powerlaw model
        popt_powerlaw,_ = curve_fit(f=powerlaw, xdata=t, ydata=y)
        ## store
        df_results.at[(pop,target), 'powerlaw_a'] = popt_powerlaw[0]
        df_results.at[(pop,target), 'powerlaw_b'] = popt_powerlaw[1]
        ## compute trajectory
        y_hat = powerlaw(t, *popt_powerlaw)
        ## store sum of residuals squared
        rss = np.power(y_hat - y,2).sum()
        rsquared = 1 - (rss/np.power(y - y.mean(),2).sum())
        df_results.at[(pop, target), 'powerlaw_rss']  = rss
        df_results.at[(pop, target), 'powerlaw_rsquared']  = rsquared
        ## plot trajectory
        param_label = 'powerlaw: a=%5.4f, b=%5.5f' % tuple(popt_powerlaw)
        sns.lineplot(x=t, y =y_hat, color = color_power,ax=ax,\
                     label = param_label + rf", $R^2={rsquared:.2f}$")
    
    
        ax.legend()
        ax.set_title(pop, loc = 'right')
        
      




    fig.savefig(FIG_DIR + f"fits_to_individual_populations_using_{target}.pdf", DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
    
  ## remove cluttering output
    if target != 'W':
        plt.close(fig)


## Calculate model comparison stats

#### Calculate BIC of the fit manually 

The BIC according to [Wikipedia](https://en.wikipedia.org/wiki/Bayesian_information_criterion#Gaussian_special_case) for the special case of a Gaussian error distribution can be calculate as 

$$
\mathrm{BIC} = n \cdot \log (\mathrm{RSS}) + k \cdot \log n  - n \cdot \log n
$$

where


- $n$ is the number of observations
- $k$ is the number of parameters estimated by the model,
- $\log$ is the natural logarithm


and


$$
\mathrm{RSS} = \sum_{i=1}^n (\hat{y} - y)^2
$$

is the residual sum of squares. Note that we can drop the last term $n*\log n$, since it is identical for all model fits in our comparison.

#### Compute likelihood ratio

The maximum of the likelihood function is given by

$$
\hat{l} = -\frac{n}{2} \log(2\pi) - \frac{n}{2}\log(\hat{\sigma}^2) - \frac{1}{2\hat{\sigma}^2}\cdot \mathrm{RSS}
$$

where $\hat{\sigma}$ is the reduced $\chi^2$-statistic (estimate of the error variance?).

According to [Wikipedia](https://en.wikipedia.org/wiki/Akaike_information_criterion#Comparison_with_least_squares)
we know that for a Gaussian distribution of errors that


$$ 
\hat{\sigma}^2 = \mathrm{RSS}/n,
$$
which we insert in the likelihood function to get

$$
\hat{l} = -\frac{n}{2} ( \log(2\pi) + 1) - \frac{n}{2}\log(\mathrm{RSS}/n) 
$$

The paper by Wiser et al. in Table S1 refers to an 'odds ratio'. It is not clear how this odds ratio is defined from the BIC. Here we assume that they mean a likelihood ratio. The likelihood ratio between two model fits with log-likelihood $\hat{l}_A$ and $\hat{l}_B$ is defined as

$$
\text{likelihood ratio}\qquad p = \frac{\exp(\hat{l}_A)}{\exp(\hat{l}_B)}
$$



Note that we are dividing the likelihoods on a linear scale $L=\exp(l)$ because these actually correspond to the probability density.

A large value of $p$ is supposed to indicate superiority of model $A$ over model $B$. Naively, this ratio can be interpreted as 

> the probability that model A is true in a world where there is only model A and B. 

#### Calculate AIC of the fit manually 

The AIC according to [Wikipedia](https://en.wikipedia.org/wiki/Akaike_information_criterion#Comparison_with_least_squares) for the special case of a Gaussian error distribution can be calculate as 

$$
\mathrm{AIC} = 2k + n \cdot \log \mathrm{RSS}  + C
$$

where


- $n$ is the number of observations
- $k$ is the number of parameters estimated by the model,
- $C$ is a constant that only depends on the number and value of datapoints,
- $\log$ is the natural logarithm


and


$$
\mathrm{RSS} = \sum_{i=1}^n (\hat{y} - y)^2
$$

is the residual sum of squares. Note that we can drop the last term $C$, since it is identical for all model fits in our comparison.

In [ ]:
## set parameters

k = 3 # include one extra parameter for variance of error distribution


for pop in list_populations:
    for target  in ['W', 's+1', 'delta_log+1']:

        ## read number of datapoints
        n = df_results.at[(pop, target),'n_datapoints']

        ## read hyperbolic model results
        rss = df_results.at[(pop, target),'hyper_rss']
        ## compute information criteria
        aic_hyper = 2*k + n*np.log(rss)
        bic_hyper = n*np.log(rss) + k*np.log(n)
        likelihood_hyper = -n/2*(np.log(2*np.pi) + 1) -n/2*np.log(rss/n)
        ## store
        df_results.at[(pop, target),'hyper_aic'] = aic_hyper
        df_results.at[(pop, target),'hyper_bic'] = bic_hyper
        df_results.at[(pop, target),'hyper_likelihood'] = likelihood_hyper

        ## read powerlaw model results
        rss = df_results.at[(pop, target),'powerlaw_rss']
        ## compute information criteria
        aic_powerlaw = 2*k + n*np.log(rss)
        bic_powerlaw = n*np.log(rss) + k*np.log(n)
        likelihood_powerlaw = -n/2*(np.log(2*np.pi) + 1) -n/2*np.log(rss/n)
        ## store
        df_results.at[(pop, target),'powerlaw_aic'] = aic_powerlaw
        df_results.at[(pop, target),'powerlaw_bic'] = bic_powerlaw
        df_results.at[(pop, target),'powerlaw_likelihood'] = likelihood_powerlaw



In [ ]:
### compute model comparison stats based on BIC

df_results['delta_bic'] = df_results['hyper_bic'] - df_results['powerlaw_bic']
df_results['likelihood_ratio'] = np.exp(df_results['powerlaw_likelihood'] - df_results['hyper_likelihood'])


In [ ]:
## coompute model comparison stats based on AIC
df_results['delta_aic'] = df_results['hyper_aic'] - df_results['powerlaw_aic']

for pop in list_populations:
    for target  in ['W', 's+1', 'delta_log+1']:
        aic_hyper = df_results.at[(pop, target), 'hyper_aic']
        aic_powerlaw = df_results.at[(pop, target), 'powerlaw_aic']

        ## identify model with smaller aic
        aic_min = np.min([aic_hyper,aic_powerlaw])
        aic_max = np.max([aic_hyper,aic_powerlaw])

        ## compute probability
        ### see https://en.wikipedia.org/wiki/Akaike_information_criterion#How_to_use_AIC_in_practice
        prob_max_model_is_better = np.exp((aic_min - aic_max)/2)


        df_results.at[(pop, target), 'akaike_pvalue'] = prob_max_model_is_better

In [ ]:
col_to_print = ['hyper_bic', 'powerlaw_bic', 'delta_bic', 'likelihood_ratio']
rows_to_print =[v for v in df_results.index if v[1] == 'W']
df_to_print = df_results.loc[rows_to_print,col_to_print].copy(deep=True)

df_to_print = df_to_print.reset_index(level = 'target', drop=True)
order = [f'Ara - {v}' for v in range(1,7)] + [f'Ara + {v}' for v in range(1,7)] 
df_to_print.loc[order]

In [ ]:
### store as excel file
## save
df_to_print.to_csv(FIG_DIR + 'reproduce_TableS2_stats_for_single_population_fits.csv', index=True)